<a href="https://colab.research.google.com/github/BlackCurrantDS/ATiML-Project/blob/master/Remove_pos_tag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
books = pd.read_csv("cleaned_books.csv")

In [0]:
import spacy

In [0]:
nlp = spacy.load('en', disable=['parser', 'ner'])

In [95]:
import gensim
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['text']]


In [0]:
def lemmatization(texts, allowed_postags=['ADJ', 'VERB', 'ADV']):
   
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return (texts_out)

In [112]:
data = {'text' : ["Priya is sick but she doesn't want to goto party She is scared Jake doesn't want to go either"]}
df = pd.DataFrame(data)
df.head()

,text
0,Priya is sick but she doesn't want to goto par...


In [115]:
df['new'] = lemmatization(df['text'].apply(word_tokenize).tolist(), allowed_postags=['ADJ','ADV'])
df['new'] = df['new'].str.join(',')
df

,text,new
0,Priya is sick but she doesn't want to goto par...,"sick,scared,either"


In [0]:
books['new'] = lemmatization(books['data'].apply(word_tokenize).tolist(), allowed_postags=['ADJ','ADV'])


In [0]:
books['new'] = books['new'].str.join(' ')  #if values are list df['Age'] = df['Age'].str.join(',') if string then strip

In [126]:
books.head(5)

,Unnamed: 0,id,genre,data,new
0,0,0,Sea and Adventure,dedicated brother allan melville long ago publ...,ago narrative reverse previous hound reelingly...
1,1,1,Sea and Adventure,voyaging straight maramma lived reigned myster...,straight reigned high proper great lofty hills...
2,2,2,Sea and Adventure,slender girl long tan coat white tamoshanter f...,sandy lazily inland great grim bustle temporar...
3,3,3,Sea and Adventure,heart great hill land glenavelin stretch west ...,wider great upland main single great real fart...
4,4,4,Sea and Adventure,yarn day clipper sailingship zenith glory reno...,medium frequently comparatively rare high true...


In [127]:
len(books['data'][0])

282864

In [128]:
len(books['new'][0])

52502

In [0]:
del books['data']

In [0]:
books.rename(columns={'new':'data'}, inplace=True)

In [139]:
books.head(5)

,Unnamed: 0,id,genre,data
0,0,0,Sea and Adventure,ago narrative reverse previous hound reelingly...
1,1,1,Sea and Adventure,straight reigned high proper great lofty hills...
2,2,2,Sea and Adventure,sandy lazily inland great grim bustle temporar...
3,3,3,Sea and Adventure,wider great upland main single great real fart...
4,4,4,Sea and Adventure,medium frequently comparatively rare high true...


In [0]:
books.to_csv("books_withoutNouns.csv")

In [97]:
df['nouns'] = df['text'].apply(lambda x : lemmatization(x, allowed_postags=['ADJ', 'VERB', 'ADV']))
df

,text,nouns
0,Priya is sick but she doesn't want to goto par...,"[[], [], [], [], [], [], [], [], [], [], [], [..."


In [0]:
data_lemmatized = lemmatization(data_words, allowed_postags=['ADJ', 'VERB','ADV'])

In [0]:
print(data_lemmatized)

[['sick', 'want', 'goto', 'scared', 'want', 'go', 'either']]


In [0]:
data_words = list(sent_to_words(books['data']))

print(data_words[:1])

print("length of the data before removing nouns is", len(data_words[:1]))

[['dedicated', 'brother', 'allan', 'melville', 'long', 'ago', 'published', 'narrative', 'voyage', 'pacific', 'quarter', 'received', 'incredulity', 'thought', 'occurred', 'writing', 'romance', 'polynesian', 'adventure', 'publishing', 'fiction', 'received', 'verity', 'degree', 'reverse', 'previous', 'experience', 'thought', 'germ', 'mardi', 'york', 'january', 'course', 'topsail', 'set', 'coralhung', 'anchor', 'swing', 'bow', 'royal', 'breeze', 'sea', 'baying', 'hound', 'spread', 'canvas', 'alow', 'side', 'stun', 'sail', 'till', 'hawk', 'pinion', 'poised', 'shadow', 'sea', 'sail', 'reelingly', 'cleave', 'brine', 'wend', 'ye', 'mariner', 'sail', 'ravavai', 'isle', 'sea', 'northward', 'tropic', 'capricorn', 'westward', 'pitcairns', 'island', 'mutineer', 'bounty', 'settled', 'ravavai', 'stepped', 'ashore', 'month', 'previous', 'embarked', 'cruise', 'whale', 'brain', 'enlightens', 'ravavai', 'sail', 'gallipagos', 'called', 'enchanted', 'island', 'reason', 'wild', 'current', 'eddy', 'met', 'ro

In [0]:
import nltk
df['noun'] = df['text'].apply(lambda x: lemmatization(x, allowed_postags=['ADJ', 'VERB','ADV']))

In [0]:
df

,text,noun
0,Priya is sick but she doesn't want to goto par...,"[[], [], [], [], [], [], [], [], [], [], [], [..."


In [0]:
print(data_lemmatized)

In [0]:
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk import RegexpParser
from nltk import Tree
import pandas as pd

def get_continuous_chunks(text, chunk_func=ne_chunk):
    chunked = chunk_func(pos_tag(word_tokenize(text)))
    continuous_chunk = []
    current_chunk = []

    for subtree in chunked:
        if type(subtree) == Tree:
            current_chunk.append(" ".join([token for token, pos in subtree.leaves()]))
        elif current_chunk:
            named_entity = " ".join(current_chunk)
            if named_entity not in continuous_chunk:
                continuous_chunk.append(named_entity)
                current_chunk = []
        else:
            continue

    return continuous_chunk

#https://stackoverflow.com/questions/49564176/python-nltk-more-efficient-way-to-extract-noun-phrases


In [0]:

df = pd.DataFrame({'text':['This is a foo, bar sentence with New York city.', 
                           'Another bar foo Washington DC thingy with Bruce Wayne.']})

df['text'].apply(lambda sent: get_continuous_chunks((sent)))

0                   [New York]
1    [Washington, Bruce Wayne]
Name: text, dtype: object

In [0]:
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk import RegexpParser
from nltk import Tree
import pandas as pd

# Defining a grammar & Parser
NP = "NP: {(<V\w+>|<NN\w?>)+.*<NN\w?>}"
chunker = RegexpParser(NP)

def get_continuous_chunks(text, chunk_func=ne_chunk):
    chunked = chunk_func(pos_tag(word_tokenize(text)))
    continuous_chunk = []
    current_chunk = []

    for subtree in chunked:
        if type(subtree) == Tree:
            current_chunk.append(" ".join([token for token, pos in subtree.leaves()]))
        elif current_chunk:
            named_entity = " ".join(current_chunk)
            if named_entity not in continuous_chunk:
                continuous_chunk.append(named_entity)
                current_chunk = []
        else:
            continue

    return continuous_chunk


df = pd.DataFrame({'text':['This is a foo, bar sentence with New York city.', 
                           'Another bar foo Washington DC thingy with Bruce Wayne.']})


df['text'].apply(lambda sent: get_continuous_chunks(sent, chunker.parse))